## Query OSM Data for VT and Build Road Network
---

Authors: Joseph Holler

Reproduction Materials Available at: https://github.com/samroubin/VTPharmacy/tree/main

Created: `2024-01-14`
Revised: `2024-01-29`

### Purpose
This notebook was used only to query OpenStreetMap data and build and save a network graph of roads in and near Vermont.
This road graph has been saved and provided with the research compendium on OSF because the CyberGISX system with public resources is too slow to query and process the data from OSM.

### Computational Environment
In order for OSMNX to work in a new environment, it is highly recommended to install with these instructions https://osmnx.readthedocs.io/en/stable/installation.html to create a new environment with the OSMNX package: 
```
conda create -n ox -c conda-forge --strict-channel-priority osmnx
```
Or create a new environment and install OSMNX as the first package. 
This notebook was run with Python version 3.12.1 and osmnx version 1.8.1.

### Modules
Import necessary libraries to run this model.
See `environment.yml` for the library versions used for this analysis.

In [ ]:
# Import modules
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
from IPython.display import display, clear_output
from shapely.ops import nearest_points   #for hospital_setting function
import warnings
import os
from shapely.geometry import Point, LineString, Polygon
import sys

warnings.filterwarnings("ignore")
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))
print("Python version:", sys.version)

## Check Directories


In [ ]:
# Check working directory
os.getcwd()

In [ ]:
# Use to set work directory properly
if os.path.basename(os.getcwd()) == 'code':
    os.chdir('../../')
os.getcwd()

### Load the Road Network

If `osm_roads.graphml` does not already exist, this cell will query the road network from OpenStreetMap.  

For very large regions, this can take several hours. On July 9, 2024, the full New England region took 4 hours and 45 minutes.

In [ ]:
%%time
# To create a new graph from OpenStreetMap, delete or rename the graph file (if it exists)
# AND set OSM to True
# This is more likely to work on a local computer than CyberGISX

# Define the place name for Vermont
places = ['Vermont, USA', 'Massachusetts, USA', 'New Hampshire, USA', 'Rhode Island, USA', 'Connecticut, USA', 'Maine, USA']

roads_path = "./data/raw/private/osm_roads.graphml"

# if buffered street network is not saved, and OSM is preferred, generate a new graph from OpenStreetMap and save it
if not os.path.exists(roads_path):
    print("Loading buffered road network from OpenStreetMap. Please wait... runtime may exceed 9min...", flush=True)
    G = ox.graph_from_place(places, network_type='drive') 
    print("Saving road network to", roads_path, " Please wait...", flush=True)
    ox.save_graphml(G, roads_path)
    print("Data saved.")
      
# load the saved network graph
if os.path.exists(roads_path):
    print("Loading road network from", roads_path, "Please wait...", flush=True)
    G = ox.load_graphml(roads_path) 
    print("Data loaded.") 
else:
    print("Error: could not load the road network from file.")

### Save geopackage of the network
This will take a long time with large graphs.

In [ ]:
ox.io.save_graph_geopackage(G, "./data/derived/private/osm_roads.gpkg")

### Check speed data
The code below converts the graph into geopandas edges (lines) and nodes (points). 
It then counts frequences of `maxspeed` key values for each edge and outputs a summary.
There is probably a better way to do this with teh graph itself rather than converting to geopandas first... as is, the runtime is slow.

In [ ]:
%%time
# Turn network edges into a geodataframe
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Count frequency of each speed value
speed_values = edges['maxspeed'].value_counts()

# Ouput number of edges and frequences of speed values
print(str(len(edges)) + " edges in graph")
print(speed_values.to_string())

### Speed and time
Correct speed limits (in kilometers per hour) and calculate travel times (in seconds)

In [ ]:
%%time
ox.speed.add_edge_speeds(G)
ox.speed.add_edge_travel_times(G)

### Plot the graph
This takes a long time to run, and I don't recommend it with very large graphs.

In [ ]:
%%time
ox.plot_graph(G, node_size = 1, bgcolor = 'white', node_color = 'black', edge_color = "#333333", node_alpha = 0.3, edge_linewidth = 0.5)

Display all the unique highway types, which are used to impute the speed limits for each category of highway.

In [ ]:
# view all highway types
print(edges['highway'].value_counts())

### Save geopackage of the network
This will take a long time with large graphs.

In [ ]:
ox.io.save_graph_geopackage(G, "./data/derived/private/osm_roads_time.gpkg")